# WCA Results - Automated Download and Import

Created by Michael George (AKA Logiqx)

## Database Parameters

Update the connection details for your MySQL environment.

In [1]:
database = "WCA"
username = "root"
password = "fredms90"

## Download the HTML

Fetch the database export  page from the WCA website.

In [2]:
# The library urllib2 will be used for the download
import urllib2
import ssl

# Specify the URL
base_url = "https://www.worldcubeassociation.org/results/misc/"

# Specify the URL
html_url = base_url + "export.html"

# Do not verify certicates
context = ssl._create_unverified_context()

# Open the URL and retrieve the TML
page = urllib2.urlopen(html_url, context=context)

## Parse the HTML

Locate the link for the ZIP file containing the SQL script.

In [3]:
# The library Beautiful Soup will be used for the HTML parsing
from bs4 import BeautifulSoup

# Parse the HTML using beautiful soup
soup = BeautifulSoup(page, "lxml")

# Find all of the links on the page
links = soup.find_all("a")

# Iterate through links
for link in links:
    
    # Get the hyperlink reference
    href = link.get("href")
    
    # If it is the SQL script it's the one that we want
    if href.endswith(".sql.zip"):
        
        # Record the SQL and exit the loop
        zip_fn = href
        zip_url = base_url + href
        break

## Download the ZIP

Save the ZIP to the local machine.

In [4]:
# Create file handle for the ZIP
infile = urllib2.urlopen(zip_url, context=context)

# Write the ZIP to a local file
with open(zip_fn, "wb") as outfile:
    outfile.write(infile.read())

# Close the URL
infile.close()

## Extract the SQL

Extract the SQL script from within the ZIP file.

In [5]:
# Use the zipfile library to handle the zipfile
import zipfile

# Open the ZIP file
zipfile = zipfile.ZipFile(zip_fn, "r")

# Iterate through members
for member in zipfile.namelist():
    
    # Is it the SQL?
    if member.endswith(".sql"):
        
        # Extract the SQL
        zipfile.extract(member)

# Close the ZIP file
zipfile.close()

## Generic SQL Function

Simple function to run a SQL script

In [2]:
# Use the OS library to execute mysql script
import os

def runSqlScript(source):   
    cmd = 'mysql --database=%s --user=%s --password=%s --execute="source %s" --default-character-set=utf8' % (database, username, password, source)
    os.system(cmd)

## Populate the WCA Database

Note: The actual database is expected to exist already

In [ ]:
runSqlScript('WCA_export.sql')

## Create Indices

Create table indices to improve query performance.

In [11]:
runSqlScript('../sql/create_indices.sql')

## Run Extracts

Extract data from database for subsequent analysis - percentiles, rankings, etc

In [4]:
runSqlScript('../sql/extract_known_details.sql')
runSqlScript('../sql/extract_senior_aggs.sql')
runSqlScript('../sql/extract_wca_aggs.sql')

# All Done!